In [7]:
from sklearn.metrics import (
    f1_score, 
    confusion_matrix, 
    make_scorer, 
    accuracy_score, 
    recall_score, 
    matthews_corrcoef
)
from sklearn.model_selection import cross_val_score, train_test_split, cross_validate
from fairlearn.metrics import (
    count,
    selection_rate,
    equalized_odds_difference,
    false_positive_rate,
    false_negative_rate,
    demographic_parity_difference,
    MetricFrame,
    true_negative_rate,
    true_positive_rate
)

from graphs import (
    create_df_ranges, 
    eval_metrics_graph, 
    fig_train_test, 
    create_df_groups_metric,
    create_df_metrics, 
    pareto_fig, 
    comparison_graph, 
    graph_eval_groups, 
    graph_eval_groups_metric,
    indicators,
    graph_opt_orig,
    graph_fair_opt_orig,
    create_df_groups_metrics
)

from metrics import (
    equality_opportunity_difference, 
    predictive_equality_difference, 
    metric_evaluation, 
    get_metric_evaluation,
    mse,
    mae,
    nmse,
    nmae
)

from fairlearn.datasets import fetch_adult
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from metrics import equality_opportunity_difference, predictive_equality_difference, metric_evaluation, get_metric_evaluation
from sklearn.utils import resample
import tqdm as notebook_tqdm

import dill
import optuna
import plotly
import numpy as np
import optuna

In [8]:
file_name = 'results/sex/recall-fpr-models-motpe-succesivehalving-parallel-180trials-1sim.pkl'
with open(file_name, 'rb') as in_strm:
    results = dill.load(in_strm)

In [9]:
sensitive_col = 'sex'

data = fetch_adult(as_frame=True)
X_raw = data.data
y = (data.target == ">50K") * 1

if sensitive_col == 'race':
    mapping = {'White':'white','Black':'black','Asian-Pac-Islander':'others','Amer-Indian-Eskimo':'others','Other':'others'}
    X_raw.loc[:,'race'] = X_raw['race'].map(mapping).astype("category")
    
A = X_raw[sensitive_col]
perc = .5
X_raw, y, A = resample(X_raw, y, A, n_samples=int(perc*X_raw.shape[0]), random_state = 123)


numeric_transformer = Pipeline(
    steps=[
        ("impute", SimpleImputer()),
        ("scaler", StandardScaler()),
    ]
)
categorical_transformer = Pipeline(
    [
        ("impute", SimpleImputer(strategy="most_frequent")),
        ("ohe", OneHotEncoder(handle_unknown="ignore")),
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, selector(dtype_exclude="category")),
        ("cat", categorical_transformer, selector(dtype_include="category")),
    ]
)

pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        (
            "classifier",
            LGBMClassifier(n_jobs=-1),
        ),
    ]
)

/home/ooti/anaconda3/envs/fairenv/lib/python3.10/site-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [10]:
#results = [results]

In [11]:
import re

metrics_sim_u = []
metrics_sim = []
res_sim = []
models_sim_u = []
models_sim = []
y_pred_sim = []
y_true_sim = []
y_pred_sim_u = []
models = [
        #LogisticRegression(), 
          RandomForestClassifier(), 
          GradientBoostingClassifier(), 
          LGBMClassifier()
         ]
for sim_n,res in enumerate(results):
    print(sim_n)
    (X_train, X_test, y_train, y_test, A_train, A_test) = train_test_split(
    X_raw, y, A, test_size=0.8, stratify=y, random_state=sim_n
    )

    X_train = X_train.reset_index(drop=True)
    X_test = X_test.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    y_test = y_test.reset_index(drop=True)
    A_train = A_train.reset_index(drop=True)
    A_test = A_test.reset_index(drop=True)

    metrics_sim_u_aux = []
    models_sim_u_aux = []
    y_pred_aux = []
    for model in models:
        print(type(model).__name__)
        models_sim_u_aux.append(type(model).__name__)
        pipeline.steps.pop(1)
        pipeline.steps.insert(1,('classifier', model))
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
        y_pred_aux.append(y_pred)
        metrics_sim_u_aux.append(
            metric_evaluation(
                        y_true=y_test, 
                        y_pred=y_pred, 
                        sensitive_features=X_test[sensitive_col]
                        )
                )
    y_pred_sim_u.append(y_pred_aux)    
    metrics_sim_u.append(metrics_sim_u_aux)
    models_sim_u.append(models_sim_u_aux)

    models_sim_aux = []
    metrics_sim_aux = []
    res_value = []
    y_pred_aux = []
    for i in range(len(res.best_trials)):
        best_params = {re.sub('^[a-z]+_','',key):value for key, value in res.best_trials[i].params.items()}
        pipeline.steps.pop(1)
        classifier_name = best_params.pop('classifier')
        models_sim_aux.append(classifier_name)
        if classifier_name == "logit": 
            pipeline.steps.insert(1,('classifier', LogisticRegression()))
        elif classifier_name =="RF":
            pipeline.steps.insert(1,('classifier', RandomForestClassifier()))
        elif classifier_name =="LGBM":
            pipeline.steps.insert(1,('classifier', LGBMClassifier()))
        elif classifier_name =="GBM":
            pipeline.steps.insert(1,('classifier', GradientBoostingClassifier()))
        pipeline['classifier'].set_params(**best_params)
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
        y_pred_aux.append(y_pred)
        res_value.append((res.best_trials[i].values[0],res.best_trials[i].values[1]))
        metrics_sim_aux.append(
            metric_evaluation(
                        y_true=y_test, 
                        y_pred=y_pred, 
                        sensitive_features=X_test[sensitive_col]
                        )
                )
    y_pred_sim.append(y_pred_aux)
    y_true_sim.append(y_test)
    models_sim.append(models_sim_aux)
    res_sim.append(res_value)
    metrics_sim.append(metrics_sim_aux)

results_dict = {}
results_dict['res_sim'] = res_sim
results_dict['models_sim'] = models_sim
results_dict['metrics_sim'] = metrics_sim
results_dict['metrics_sim_u'] = metrics_sim_u
results_dict['models_sim_u'] = models_sim_u
results_dict['y_pred_sim_u'] = y_pred_sim_u
#results_dict['y_true_sim_u'] = y_true_sim_u
results_dict['y_pred_sim'] = y_pred_sim
results_dict['y_true_sim'] = y_true_sim

file = file_name[:-4] + '-results.pkl' 
with open(file, 'wb') as file:
    dill.dump(results_dict, file)
    print(f'Object successfully saved to "{file}"')


0
RandomForestClassifier
GradientBoostingClassifier
LGBMClassifier


/home/ooti/anaconda3/envs/fairenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ooti/anaconda3/envs/fairenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ooti/anaconda3/envs/fairenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ooti/anaconda3/envs/fairenv/lib/python3.10/site-pack

Object successfully saved to "<_io.BufferedWriter name='results/sex/recall-fpr-models-motpe-succesivehalving-parallel-180trials-1sim-results.pkl'>"
